<a href="https://colab.research.google.com/github/GGoYoungHee/RGBrand/blob/main/Kobayashi_Color_Adjective_Mapping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

데이터

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd

### 형용사

adj = pd.read_csv('/content/drive/My Drive/ADV/형용사/형용사 번역_fin.csv')
adj.head()

,shape id,sh_ko,point id,po_kor,x,y,shape point
0,casual,격의 없는,amusing,재미있는,218,687,casual
1,casual,격의 없는,bright,선명한,169,752,casual
2,casual,격의 없는,casual,격의 없는,154,651,casual
3,casual,격의 없는,cheerful,명랑한,112,859,casual
4,casual,격의 없는,colorful,알록달록한,218,572,casual


In [ ]:
### 색상표

rgb = pd.read_csv('/content/drive/My Drive/ADV/형용사/최종_위치찍기 with RGB.csv',encoding='cp949')
rgb['point id'] = range(1,131)
rgb = rgb[['point id','R','G','B','X','Y']]
rgb.head()

,point id,R,G,B,X,Y
0,1,236,217,202,594,1048
1,2,245,223,182,659,1081
2,3,249,239,189,722,1070
3,4,228,235,191,765,1049
4,5,221,232,207,834,1026


In [ ]:
### 인터브랜드

data = pd.read_csv('/content/drive/My Drive/ADV/형용사/최종데이터.csv',encoding='cp949')
data = data[['Table','name','sector','Img','R1','G1','B1','H1','S1','V1','hex1']]
data.head()

,Table,name,sector,Img,R1,G1,B1,H1,S1,V1,hex1
0,Interbrand,3M,일용소비재,https://www.interbrand.com/assets/00000001482.png,237,32.0,35,359.12195,0.86498,0.92941,#ed2023
1,Interbrand,Absolut,식음료,https://www.interbrand.com/assets/00000000005.png,14,80.0,157,212.30769,0.91083,0.61569,#0e509d
2,Interbrand,Accenture,사업서비스,https://www.interbrand.com/assets/00000001483.png,0,0.0,0,0.00000,0.00000,0.00000,#000000
3,Interbrand,adidas,의류,https://www.interbrand.com/assets/00000001484.png,0,0.0,0,0.00000,0.00000,0.00000,#000000
4,Interbrand,Adobe,사업서비스,https://www.interbrand.com/assets/00000001485.png,78,0.0,0,0.00000,1.00000,0.30588,#4e0000


In [ ]:
data.iloc[534,:]

Table                                                Jungso
name                                              퓨쳐스트림네트웍스
sector                                                IT서비스
Img       https://d1vv4p3ixxyri0.cloudfront.net/companie...
R1                                                       14
G1                                                      NaN
B1                                                       14
H1                                                        0
S1                                                        0
V1                                                   0.0549
hex1                                                #0e0e0e
Name: 534, dtype: object

In [ ]:
data = data.drop(534)

In [ ]:
data = data.astype({'G1': 'int'})
data = data.reset_index()

기업 로고 rgb를 기준) KNN으로 가장 가까운 색상표 상의 색상에 할당

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier(n_neighbors = 1)

In [ ]:
x_train = rgb[['R','G','B']]
y_train = rgb['point id']

In [ ]:
knn.fit(x_train,y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=1, p=2,
                     weights='uniform')

In [ ]:
data['knn predict'] = knn.predict(data[['R1','G1','B1']])
data = data.drop(columns='index')
data

,Table,name,sector,Img,R1,G1,B1,H1,S1,V1,hex1,knn predict
0,Interbrand,3M,일용소비재,https://www.interbrand.com/assets/00000001482.png,237,32,35,359.12195,0.86498,0.92941,#ed2023,41
1,Interbrand,Absolut,식음료,https://www.interbrand.com/assets/00000000005.png,14,80,157,212.30769,0.91083,0.61569,#0e509d,48
2,Interbrand,Accenture,사업서비스,https://www.interbrand.com/assets/00000001483.png,0,0,0,0.00000,0.00000,0.00000,#000000,130
3,Interbrand,adidas,의류,https://www.interbrand.com/assets/00000001484.png,0,0,0,0.00000,0.00000,0.00000,#000000,130
4,Interbrand,Adobe,사업서비스,https://www.interbrand.com/assets/00000001485.png,78,0,0,0.00000,1.00000,0.30588,#4e0000,101
...,...,...,...,...,...,...,...,...,...,...,...,...
535,Jungso,피투피시스템즈,사업서비스,https://d1vv4p3ixxyri0.cloudfront.net/companie...,199,21,66,344.83146,0.89447,0.78039,#c71542,61
536,Jungso,피플펀드컴퍼니,금융서비스,https://d1vv4p3ixxyri0.cloudfront.net/companie...,38,161,201,194.72393,0.81095,0.78824,#26a1c9,38
537,Jungso,하이파킹,운송,https://d1vv4p3ixxyri0.cloudfront.net/companie...,27,204,124,152.88136,0.86765,0.80000,#1bcc7c,36
538,Jungso,해브앤비,일용소비재,https://d1vv4p3ixxyri0.cloudfront.net/companie...,60,51,49,10.90909,0.18333,0.23529,#3c3331,110


In [ ]:
rgb[rgb['point id'] == 130]
# 로고가 검은색인 기업은 r,g,b값이 10,10,10인 회색의 색상에 할당되는 것을 확인

,point id,R,G,B,X,Y
129,130,10,10,10,715,3


데이터 조인

In [ ]:
mer = pd.merge(data,rgb,how='left',left_on='knn predict', right_on='point id')
mer.head()

,Table,name,sector,Img,R1,G1,B1,H1,S1,V1,hex1,knn predict,point id,R,G,B,X,Y
0,Interbrand,3M,일용소비재,https://www.interbrand.com/assets/00000001482.png,237,32,35,359.12195,0.86498,0.92941,#ed2023,41,41,231,47,39,64,719
1,Interbrand,Absolut,식음료,https://www.interbrand.com/assets/00000000005.png,14,80,157,212.30769,0.91083,0.61569,#0e509d,48,48,3,86,155,721,331
2,Interbrand,Accenture,사업서비스,https://www.interbrand.com/assets/00000001483.png,0,0,0,0.00000,0.00000,0.00000,#000000,130,130,10,10,10,715,3
3,Interbrand,adidas,의류,https://www.interbrand.com/assets/00000001484.png,0,0,0,0.00000,0.00000,0.00000,#000000,130,130,10,10,10,715,3
4,Interbrand,Adobe,사업서비스,https://www.interbrand.com/assets/00000001485.png,78,0,0,0.00000,1.00000,0.30588,#4e0000,101,101,79,46,43,552,51


color & 형용사 매핑

거리순으로 형용사와 색상 매칭

In [ ]:
from scipy.spatial import distance

* 형용사별 가장 거리가 가까운 기업 *

In [ ]:
adj.head()

# 형용사 데이터 좌표

,shape id,sh_ko,point id,po_kor,x,y,shape point
0,casual,격의 없는,amusing,재미있는,218,687,casual
1,casual,격의 없는,bright,선명한,169,752,casual
2,casual,격의 없는,casual,격의 없는,154,651,casual
3,casual,격의 없는,cheerful,명랑한,112,859,casual
4,casual,격의 없는,colorful,알록달록한,218,572,casual


In [ ]:
mer.head()

# 색상 데이터 좌표

,Table,name,sector,Img,R1,G1,B1,H1,S1,V1,hex1,knn predict,point id,R,G,B,X,Y
0,Interbrand,3M,일용소비재,https://www.interbrand.com/assets/00000001482.png,237,32,35,359.12195,0.86498,0.92941,#ed2023,41,41,231,47,39,64,719
1,Interbrand,Absolut,식음료,https://www.interbrand.com/assets/00000000005.png,14,80,157,212.30769,0.91083,0.61569,#0e509d,48,48,3,86,155,721,331
2,Interbrand,Accenture,사업서비스,https://www.interbrand.com/assets/00000001483.png,0,0,0,0.00000,0.00000,0.00000,#000000,130,130,10,10,10,715,3
3,Interbrand,adidas,의류,https://www.interbrand.com/assets/00000001484.png,0,0,0,0.00000,0.00000,0.00000,#000000,130,130,10,10,10,715,3
4,Interbrand,Adobe,사업서비스,https://www.interbrand.com/assets/00000001485.png,78,0,0,0.00000,1.00000,0.30588,#4e0000,101,101,79,46,43,552,51


In [ ]:
adjective = []
minimum = []
for i in range(len(mer)):
  a_xy = mer.loc[i,['X','Y']]            # 기업의 x,y와
  dis = []
  for j in range(len(adj)) :
    b_xy = adj.loc[j,['x','y']]          # 모든 색상의 x,y의 유클리디안 거리를 계산
    dis.append(distance.euclidean(a_xy, b_xy))
  minindex = dis.index(min(dis))
  adjective.append(adj.loc[minindex,'point id'])
  minimum.append(min(dis))

In [ ]:
res1 = pd.concat([mer,pd.DataFrame({'adjective':adjective,'distance':minimum})],axis=1)
res1
fin = pd.merge(res1,adj,left_on='adjective',right_on='point id',how='left')
#fin.to_csv('alldata.csv',index=False)
fin.head()

,Table,name,sector,Img,R1,G1,B1,H1,S1,V1,hex1,knn predict,point id_x,R,G,B,X,Y,adjective,distance,shape id,sh_ko,point id_y,po_kor,x,y,shape point
0,Interbrand,3M,일용소비재,https://www.interbrand.com/assets/00000001482.png,237,32,35,359.12195,0.86498,0.92941,#ed2023,41,41,231,47,39,64,719,dazzling,26.419690,casual,격의 없는,dazzling,빛나는,87,732,casual
1,Interbrand,Absolut,식음료,https://www.interbrand.com/assets/00000000005.png,14,80,157,212.30769,0.91083,0.61569,#0e509d,48,48,3,86,155,721,331,stylish,27.802878,chic,시크한,stylish,멋진,704,353,chic
2,Interbrand,Accenture,사업서비스,https://www.interbrand.com/assets/00000001483.png,0,0,0,0.00000,0.00000,0.00000,#000000,130,130,10,10,10,715,3,dignified,95.047357,dandy,멋있는,dignified,위엄 있는,712,98,dandy
3,Interbrand,Accenture,사업서비스,https://www.interbrand.com/assets/00000001483.png,0,0,0,0.00000,0.00000,0.00000,#000000,130,130,10,10,10,715,3,dignified,95.047357,formal,격식을 차린,dignified,위엄 있는,712,98,formal
4,Interbrand,adidas,의류,https://www.interbrand.com/assets/00000001484.png,0,0,0,0.00000,0.00000,0.00000,#000000,130,130,10,10,10,715,3,dignified,95.047357,dandy,멋있는,dignified,위엄 있는,712,98,dandy


In [ ]:
fin_50 = fin[fin['distance'] <= 50]
fin_50.to_csv('findata_under50.csv')

In [ ]:
import pandas as pd
fin = pd.read_csv('/content/findata_under50.csv')
fin.head()

,Unnamed: 0,Table,name,sector,Img,R1,G1,B1,H1,S1,V1,hex1,knn predict,point id_x,R,G,B,X,Y,adjective,distance,shape id,sh_ko,point id_y,po_kor,x,y,shape point
0,0,Interbrand,3M,일용소비재,https://www.interbrand.com/assets/00000001482.png,237,32,35,359.12195,0.86498,0.92941,#ed2023,41,41,231,47,39,64,719,dazzling,26.419690,casual,격의 없는,dazzling,빛나는,87,732,casual
1,1,Interbrand,Absolut,식음료,https://www.interbrand.com/assets/00000000005.png,14,80,157,212.30769,0.91083,0.61569,#0e509d,48,48,3,86,155,721,331,stylish,27.802878,chic,시크한,stylish,멋진,704,353,chic
2,7,Interbrand,AIG,금융서비스,https://www.interbrand.com/assets/00000000009.png,46,173,224,197.19101,0.79464,0.87843,#2eade0,38,38,59,130,157,775,683,polished,8.544004,cool-casual,밝고 선명한,polished,세련된,772,675,cool-casual
3,8,Interbrand,AIG,금융서비스,https://www.interbrand.com/assets/00000000009.png,46,173,224,197.19101,0.79464,0.87843,#2eade0,38,38,59,130,157,775,683,polished,8.544004,elegant,기품이 있는,polished,세련된,772,675,elegant
4,12,Interbrand,American Express,금융서비스,https://www.interbrand.com/assets/00000001488.png,3,112,207,207.94118,0.98551,0.81176,#0370cf,48,48,3,86,155,721,331,stylish,27.802878,chic,시크한,stylish,멋진,704,353,chic


 S * V 값에 따라 그룹 분류

In [ ]:
fin['SVmul'] = fin['S1'] * fin['V1']
fin['SVmul']

0      0.803921
1      0.560789
2      0.698036
3      0.698036
4      0.799998
         ...   
398    0.307200
399    0.698035
400    0.639223
401    0.639223
402    0.043136
Name: SVmul, Length: 403, dtype: float64

In [ ]:
fin['5group'] = pd.qcut(fin['SVmul'], 5,duplicates='drop')
fin.groupby('5group')['name'].size()

5group
(-0.001, 0.243]    81
(0.243, 0.477]     80
(0.477, 0.62]      81
(0.62, 0.761]      80
(0.761, 1.0]       81
Name: name, dtype: int64

In [ ]:
fin.loc[(fin['SVmul']<0.1) ,'5_group'] = 1
fin.loc[(fin['SVmul']>=0.1) & (fin['SVmul']<0.4) ,'5_group'] = 2
fin.loc[(fin['SVmul']>=0.4) & (fin['SVmul']<0.6) ,'5_group'] = 3
fin.loc[(fin['SVmul']>=0.6) & (fin['SVmul']<0.7) ,'5_group'] = 4
fin.loc[fin['SVmul']>=0.7,'5_group'] = 5

fin.groupby(['5_group'])['name'].size()

5_group
1.0     52
2.0     76
3.0     95
4.0     62
5.0    118
Name: name, dtype: int64

그룹별 h 값에 따라 순위 부여

In [ ]:
fin = fin.sort_values(by='H1',ascending=False).reset_index()

fin['id'] = pd.Series()

num = 1
for i in range(len(fin)):
  if fin.loc[i,'5_group'] == 1:
    fin.loc[i,'id'] = num
    num = num + 1

num = 1
for i in range(len(fin)):
  if fin.loc[i,'5_group'] == 2:
    fin.loc[i,'id'] = num
    num = num + 1

num = 1
for i in range(len(fin)):
  if fin.loc[i,'5_group'] == 3:
    fin.loc[i,'id'] = num
    num = num + 1

num = 1
for i in range(len(fin)):
  if fin.loc[i,'5_group'] == 4:
    fin.loc[i,'id'] = num
    num = num + 1

num = 1
for i in range(len(fin)):
  if fin.loc[i,'5_group'] == 5:
    fin.loc[i,'id'] = num
    num = num + 1


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  This is separate from the ipykernel package so we can avoid doing imports until


In [ ]:
fin['row'] = range(len(fin))
fin.to_csv('findata_under50_radial.csv',encoding='cp949')
fin.head()

,index,Unnamed: 0,Table,name,sector,Img,R1,G1,B1,H1,S1,V1,hex1,knn predict,point id_x,R,G,B,X,Y,adjective,distance,shape id,sh_ko,point id_y,po_kor,x,y,shape point,SVmul,5group,5_group,id,row
0,158,310,Interbrand,UBS,금융서비스,https://www.interbrand.com/assets/00000000157.png,238,34,35,359.70588,0.85714,0.93333,#ee2223,41,41,231,47,39,64,719,dazzling,26.419690,casual,격의 없는,dazzling,빛나는,87,732,casual,0.799994,"(0.761, 1.0]",5.0,1.0,0
1,13,47,Interbrand,Campbell's,식음료,https://www.interbrand.com/assets/00000000035.png,193,25,26,359.64286,0.87047,0.75686,#c1191a,91,91,172,36,48,270,335,luxurious,40.804412,gorgeous,호화로운,luxurious,호화로운,303,311,gorgeous,0.658824,"(0.62, 0.761]",4.0,1.0,1
2,0,0,Interbrand,3M,일용소비재,https://www.interbrand.com/assets/00000001482.png,237,32,35,359.12195,0.86498,0.92941,#ed2023,41,41,231,47,39,64,719,dazzling,26.419690,casual,격의 없는,dazzling,빛나는,87,732,casual,0.803921,"(0.761, 1.0]",5.0,2.0,2
3,88,193,Interbrand,LEGO,일용소비재,https://www.interbrand.com/assets/00000001540.png,243,7,11,358.98305,0.97119,0.95294,#f3070b,41,41,231,47,39,64,719,dazzling,26.419690,casual,격의 없는,dazzling,빛나는,87,732,casual,0.925486,"(0.761, 1.0]",5.0,3.0,3
4,52,126,Interbrand,Heinz,식음료,https://www.interbrand.com/assets/00000000072.png,238,36,40,358.81188,0.84874,0.93333,#ee2428,41,41,231,47,39,64,719,dazzling,26.419690,casual,격의 없는,dazzling,빛나는,87,732,casual,0.792155,"(0.761, 1.0]",5.0,4.0,4


In [ ]:
fin[['hex1']].to_csv('findata_hexcode.csv',index=False)

색상보여주기 hexcode

In [ ]:
import pandas as pd
he = pd.read_csv('/content/findata_under50_radial.csv', encoding='cp949')
he.head()

,Unnamed: 0,index,Unnamed: 0.1,Table,name,sector,Img,R1,G1,B1,H1,S1,V1,hex1,knn predict,point id_x,R,G,B,X,Y,adjective,distance,shape id,sh_ko,point id_y,po_kor,x,y,shape point,SVmul,5group,5_group,id,row
0,0,158,310,Interbrand,UBS,금융서비스,https://www.interbrand.com/assets/00000000157.png,238,34,35,359.70588,0.85714,0.93333,#ee2223,41,41,231,47,39,64,719,dazzling,26.419690,casual,격의 없는,dazzling,빛나는,87,732,casual,0.799994,"(0.761, 1.0]",5.0,1.0,0
1,1,13,47,Interbrand,Campbell's,식음료,https://www.interbrand.com/assets/00000000035.png,193,25,26,359.64286,0.87047,0.75686,#c1191a,91,91,172,36,48,270,335,luxurious,40.804412,gorgeous,호화로운,luxurious,호화로운,303,311,gorgeous,0.658824,"(0.62, 0.761]",4.0,1.0,1
2,2,0,0,Interbrand,3M,일용소비재,https://www.interbrand.com/assets/00000001482.png,237,32,35,359.12195,0.86498,0.92941,#ed2023,41,41,231,47,39,64,719,dazzling,26.419690,casual,격의 없는,dazzling,빛나는,87,732,casual,0.803921,"(0.761, 1.0]",5.0,2.0,2
3,3,88,193,Interbrand,LEGO,일용소비재,https://www.interbrand.com/assets/00000001540.png,243,7,11,358.98305,0.97119,0.95294,#f3070b,41,41,231,47,39,64,719,dazzling,26.419690,casual,격의 없는,dazzling,빛나는,87,732,casual,0.925486,"(0.761, 1.0]",5.0,3.0,3
4,4,52,126,Interbrand,Heinz,식음료,https://www.interbrand.com/assets/00000000072.png,238,36,40,358.81188,0.84874,0.93333,#ee2428,41,41,231,47,39,64,719,dazzling,26.419690,casual,격의 없는,dazzling,빛나는,87,732,casual,0.792155,"(0.761, 1.0]",5.0,4.0,4


In [ ]:
def rgb_to_hexcode(r,g,b):
    r,g,b=int(r),int(g),int(b)
    return hex(r)[2:].zfill(2)+hex(g)[2:].zfill(2)+hex(b)[2:].zfill(2)

In [ ]:
he_code = []
for i in range(len(he)):
    he_code.append(rgb_to_hexcode(he.loc[i,'R'],he.loc[i,'G'],he.loc[i,'B']))

In [ ]:
show_he = pd.concat([he,pd.DataFrame(he_code)],axis=1)[['po_kor',0]]
show_he.columns = ['po_kor','hexcode']
show_he = show_he.drop_duplicates()
show_he = show_he.drop(17)

In [ ]:
show_he.to_csv('finshow_hexcode.csv',index=False)

In [ ]:
import pandas as pd
data1 = pd.read_csv('/content/drive/My Drive/ADV/형용사/alldata.csv')
data2 = pd.read_csv('/content/drive/My Drive/ADV/형용사/alldata_under50.csv')
data1.head()

,Table,name,sector,Img,R1,G1,B1,H1,S1,V1,hex1,knn predict,point id_x,R,G,B,X,Y,adjective,distance,shape id,sh_ko,point id_y,po_kor,x,y,shape point
0,Interbrand,3M,일용소비재,https://www.interbrand.com/assets/00000001482.png,237,32,35,359.12195,0.86498,0.92941,#ed2023,41,41,231,47,39,64,719,dazzling,26.419690,casual,격의 없는,dazzling,눈부시다,87,732,casual
1,Interbrand,Absolut,식음료,https://www.interbrand.com/assets/00000000005.png,14,80,157,212.30769,0.91083,0.61569,#0e509d,48,48,3,86,155,721,331,stylish,27.802878,chic,시크하다,stylish,멋진,704,353,chic
2,Interbrand,Accenture,사업서비스,https://www.interbrand.com/assets/00000001483.png,0,0,0,0.00000,0.00000,0.00000,#000000,130,130,10,10,10,715,3,dignified,95.047357,dandy,댄디한,dignified,위엄 있는,712,98,dandy
3,Interbrand,Accenture,사업서비스,https://www.interbrand.com/assets/00000001483.png,0,0,0,0.00000,0.00000,0.00000,#000000,130,130,10,10,10,715,3,dignified,95.047357,formal,격식을 차린,dignified,위엄 있는,712,98,formal
4,Interbrand,adidas,의류,https://www.interbrand.com/assets/00000001484.png,0,0,0,0.00000,0.00000,0.00000,#000000,130,130,10,10,10,715,3,dignified,95.047357,dandy,댄디한,dignified,위엄 있는,712,98,dandy


In [ ]:
data1[['name','R1','G1','B1','adjective','distance']]

,name,R1,G1,B1,adjective,distance
0,3M,237,32,35,dazzling,26.419690
1,Absolut,14,80,157,stylish,27.802878
2,Accenture,0,0,0,dignified,95.047357
3,Accenture,0,0,0,dignified,95.047357
4,adidas,0,0,0,dignified,95.047357
...,...,...,...,...,...,...
680,피플펀드컴퍼니,38,161,201,polished,8.544004
681,하이파킹,27,204,124,clean and fresh,58.549125
682,해브앤비,60,51,49,serious,15.652476
683,휴맥스모빌리티,35,33,36,dignified,54.424259


In [ ]:
data2[['name','R1','G1','B1','adjective','distance']]

,name,R1,G1,B1,adjective,distance
0,3M,237,32,35,dazzling,26.419690
1,Absolut,14,80,157,stylish,27.802878
2,AIG,46,173,224,polished,8.544004
3,AIG,46,173,224,polished,8.544004
4,American Express,3,112,207,stylish,27.802878
...,...,...,...,...,...,...
398,피스컬노트,113,44,122,provincial,43.416587
399,피투피시스템즈,199,21,66,casual,33.615473
400,피플펀드컴퍼니,38,161,201,polished,8.544004
401,피플펀드컴퍼니,38,161,201,polished,8.544004
